# Code for Publication "Explainable Data-Driven Hypertension Identification Using Inpatient EMR Clinical Notes"

# Import modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, KFold
from sklearn import tree
import graphviz 
import xgboost as xgb

import shap

import re
import datetime
import pickle
import xml.etree.ElementTree as ET
from os import listdir
from itertools import islice

# Process documents with cTAKES and parse output 

## Process Documents with cTAKES

In [ ]:
#Specifications to pass to cTAKES

#Your UMLS credentials
username = "***Your UMLS Username***"
password = "***Your UMLS Password***"

#Path to cTAKES clinical pipeline ***REPLACE WITH YOUR PATH***
pipeline = "~/Resources/apache-ctakes-4.0.0/bin/runClinicalPipeline.sh"

#Folder containing the documents you want processed, to simplify linkage I used the chart number (RHRN) 
#as the document name: RHRN.txt
inFolder = " -i data/Text/DocType"

#Folder to put the annotated documents in
outFolder = " --xmiOut data/cTAKESoutput/DocType"

#UMLS credentials
UMLScred = f" --user {username} --pass {password}!"

#Final Shell command to execute
cmd = pipeline + inFolder + outFolder + UMLScred

!$cmd

## Parse cTAKES output 

- create a dataframe where each row is a document
- each column is a CUI, except the first which is a patient identifier (RHRN)
- each element (outside the first column) is the number of times a given CUI appeared in the document, non-negated and referring to the patient

These are then compiled into a new dataframe where each row is a patient, and each column is a CUI or Document-CUI pair. Each element is the total number of times that CUI appeared in the first case, or total number of times that CUI appeared in that document type in the second case.

In [ ]:
floc = "data/DischargeSummaries/cTAKESoutput/DocType"
cDocs = listdir(floc)
boc = pd.DataFrame()

for count, doc in enumerate(cDocs):
    tree = ET.parse(floc + doc) 
    root = tree.getroot()
        
    RHRN = doc.split('.')[0]
    
#     boc.loc[count] = 0
#     boc['RHRN'].loc[count] =RHRN
    
    concepts = root.findall(".//*[@ontologyConceptArr]")
    concepts = [x.attrib for x in concepts]
    concepts = pd.DataFrame(concepts)
    concepts = concepts[(concepts['subject'] == 'patient') & (concepts['polarity'] == '1')& (concepts['uncertainty']=='0')]
    
    cuis = root.findall(".//*[@cui]")
    tmp = {'RHRN':RHRN}
    for cui in cuis:
        concept_id = cui.attrib['{http://www.omg.org/XMI}id']
        if (concepts['ontologyConceptArr'].str.contains(concept_id).sum()) > 0:
            ind = cui.attrib['cui']
            try:
                tmp[ind]+=1
            except:
                tmp[ind] = 1
            
        
    boc = boc.append(tmp,ignore_index=True)
    
    if (count % 50)==0:
        print(count)

# Merge cTAKES labels with DAD and Chart Review data

## Merge output with DAD

In [ ]:
dad = pd.read_csv('data/DAD.csv',low_memory=False)
dad['RHRN'] = dad['RHRN'].astype(str)
boc = boc.merge(dad,on='RHRN')

## Merge with Chart review

In [ ]:
cr = pd.read_excel('data/ChartRev.xlsx',sheet_name='FULLDATA')
boc = boc.merge(cr,on='RHRN')

### Remove uncertain hypertension cases 

In [ ]:
boc =boc.loc[boc['Hypertension present'] != 'Maybe']

### Modify labels so 1 = Hypertension present and 0 = Hypertension not present

In [ ]:
boc['Hypertension present'] = (boc['Hypertension present'] == 'Yes').astype(int)

# Analyze Data

## Define some helper functions

In [ ]:
#always round .5 up
import decimal
context = decimal.getcontext()
context.rounding = decimal.ROUND_HALF_UP

In [ ]:
def format_out(func):
    def standard_round(*args):
        out = [int(round(decimal.Decimal(x*100), 0)) for x in func(*args)]
        return f"{out[0]}({out[0]-out[1]}-{out[0]+out[1]})"
    return standard_round

@format_out
def precision(predicted,actual):
    predicted = np.array(predicted)
    actual = np.array(actual)
    p = sum(np.where(predicted & actual,1,0))/sum(predicted)
    return p, ci(p,sum(predicted))

@format_out
def recall(predicted,actual):
    predicted = np.array(predicted)
    actual = np.array(actual)
    r = sum(np.where(predicted & actual,1,0))/sum(actual)
    return r, ci(r,sum(actual))

@format_out
def accuracy(predicted,actual):
    predicted = np.array(predicted)
    actual = np.array(actual)
    a = sum(np.where(predicted==actual,1,0))/len(actual)
    return a, ci(a,len(actual))

@format_out
def specificity(predicted,actual):
    predicted = np.array(predicted)
    predicted = 1 - predicted
    actual = np.array(actual)
    actual = 1 - actual
    s = sum(np.where(predicted & actual,1,0))/sum(actual)
    return s, ci(s,sum(actual))

@format_out
def NPV(predicted,actual):
    predicted = np.array(predicted)
    predicted = 1 - predicted
    actual = np.array(actual)
    actual = 1 - actual
    n = sum(np.where(predicted & actual,1,0))/sum(predicted)
    return n, ci(n,sum(predicted))

# 95% confidence intervals
def ci(p,N):
    return 1.96*(p*(1-p)/N)**0.5


def stats(predicted,actual):
    print("Sample Size = ", len(actual))
    print("Positive Cases = ",sum(actual))
    print("Cases Labeled Positive = ", sum(predicted))
#     print("Negative Cases = ", sum(1-actual))
#     print("Cases Labeled Negative = ", sum(1-predicted))    
    
    print("Recall = ",recall(predicted,actual))
    print("Specificity = ",specificity(predicted,actual))    
    print("Precision = ",precision(predicted,actual))
    print("NPV = ",NPV(predicted,actual))    
    print("Accuracy = ",accuracy(predicted,actual))    

## Create an ICD-10 case defintion column

In [ ]:
# Select out all columns with ICD diagnoses codes
cols = boc.columns
codes = cols[cols.str.contains('dxcode',flags=re.IGNORECASE)]

# Create a column containing all diagnoes codes concatenated together
boc['AllCodes']=boc[codes].fillna("").apply(lambda row: " ".join([str(x) for x in row]),axis=1)

# Create a column that equal 1 if it satisfies the ICD-10 hypertension definition and 0 otherwise
boc['ICD Hypt'] = (boc['AllCodes'].str.contains('I10|I11|I12|I13|I15')).astype(int)

## Feature selection with XGBoost

- This process is applied first on all Document types separately, and then using the top features from each model

### Select all CUIs as predictors

In [ ]:
cuis = boc.columns.str.contains('^C\d')

X = boc.loc[:,cuis]
Y = boc['Hypertension present']

### Split off training and testing sets 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, stratify=Y_all, random_state=42)

### Specify parameters

- this was performed on a GPU cluster, and the tree_method and predictor should be changed to run on a cpu

In [ ]:
xgb_model = xgb.XGBClassifier()

parameters = {
    'lambda':[0,0.5,1], #L2 regularization term on weights
    'alpha':[0,0.5,1], #L1 regularization term on weights 
    'learning_rate': [0.05], #so called `eta` value
    'objective':['binary:logistic'],
    'max_depth': [5,8],
    'min_child_weight': [4,8],
    'subsample': [0.8],
    'colsample_bytree': [0.7],
    'n_estimators': [500,1000], #number of trees, change it to 1000 for better results
    'seed': [42],
    'tree_method': ['gpu_hist'], 
    "predictor":['gpu_predictor']
}

scoring = 'roc_auc'
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

### Create Cross-Validated XGBoost models

In [ ]:
for fold, indices in enumerate(skf.split(X_train,Y_train)):
    
    print('Starting fold:',fold)
    
    clf = GridSearchCV(xgb_model, parameters, 
                       cv=5, 
                       scoring=scoring,
                       verbose=2, refit=True)

    X_samp = X_train.iloc[indices[0]]
    Y_samp = Y_train.iloc[indices[0],0]
    eval_set = [(X_train.iloc[indices[1]], Y_train.iloc[indices[1],0])]
    
    clf.fit(X_samp, Y_samp,eval_set=eval_set, early_stopping_rounds=10)
    filename = f"../Models/Hypt-Model-FOLD-{fold}-CaseIdent-JustCUIs-XGBoost-AllCUIs-AllDocs-SubjNeg.pkl"
    pickle.dump(clf, open(filename, 'wb'))

    print('Finished fold:',fold)
    

### Get stats for all folds and get top features

In [ ]:
nf = 20
top_intersect = set()
for fold, indices in enumerate(skf.split(X_train,Y_train)):
    print(f'Fold {fold} results', 42*'#')
    try:
        clf = pickle.load(open(f"../Models/Hypt-Model-FOLD-{fold}-CaseIdent-JustCUIs-XGBoost-AllCUIs-AllDocs-SubjNeg.pkl",'rb'))
    except:
        print(f'Fold {fold} did not finish',42*'@')
        continue
        
    for i in range(2): 
        pred = clf.predict(X_train.iloc[indices[i]])
        stats(pred,Y_train['hypertension_present'].iloc[indices[i]])
        print(30*'*')
        
    tg = clf.best_estimator_.get_booster().get_score(importance_type= "gain")
    tg = pd.Series(tg)
    tg.sort_values(ascending=False,inplace=True)
    top_features = tg.iloc[:nf].index        
    
    if fold == 0:
        top_intersect = set(top_features)
    else:
        top_intersect= top_intersect.intersection(set(top_features)) 


### Compare feature importances

In [ ]:
%matplotlib inline

In [ ]:
folds = 5

nf = 20

tfs = {}

for i in range(folds):
    filename = f'models/Hypt-Model-FOLD-{i}-CaseIdent-XGBoost-AllCUIs-SubjNegUncer.pkl'
    clf = pickle.load(open(filename, 'rb'))    
    
    tg = clf.best_estimator_.get_booster().get_score(importance_type= "gain")
    tg = pd.Series(tg)
    tg.sort_values(ascending=False,inplace=True)
    top_features = tg.iloc[:nf]
    # top_features

    tfs[i] = set(tg.index[:nf])

    pos = np.arange(top_features.shape[0])
    plt.subplot(folds, 1, i+1)
    plt.bar(pos,top_features)
#     plt.xticks(pos,top_features.index,rotation=-45)
    plt.ylabel("Feature Importance")
    plt.xlabel("Feature Rank")

## Examine SHAP Values

In [ ]:
shap.initjs()

In [ ]:
model = clf.best_estimator_
explainer = shap.TreeExplainer(model)

vals = X_train
shap_values = explainer.shap_values(vals)

In [ ]:
shap.summary_plot(shap_values,vals)